In [ ]:
%%capture

!pip install gdown gpxpy

In [ ]:
%%capture

import gdown

FILE_URL = 'https://drive.google.com/uc?id=1nqGQiFVmub2mDNCx0kcLp1uoRXi19qLg'
gdown.download(FILE_URL, output='gnss-logger-fix.csv', quiet=True)

In [ ]:
import pandas as pd

gnss_logger_df = pd.read_csv('/content/gnss-logger-fix.csv')
gnss_logger_df['Time'] = pd.to_datetime(gnss_logger_df['UnixTimeMillis'], unit='ms', utc=True)
gnss_logger_df.head()

In [ ]:
%%capture

import gdown

FILE_URL = 'https://drive.google.com/uc?id=1exVZ8rXLS1EpmZcbzsdIPyc2IkKb7yet'
gdown.download(FILE_URL, output='osmtracker.gpx', quiet=True)

In [ ]:
import gpxpy
import pandas as pd

def parse_gpx(filepath):
  gpx_file = open(filepath, 'r')
  gpx = gpxpy.parse(gpx_file)

  data = []

  for track in gpx.tracks:
    for segment in track.segments:
      for point in segment.points:
        time, lat, lng = point.time, point.latitude, point.longitude,
        data.append({
          'time': time,
          'lat': lat,
          'lon': lng,
        })

  df = pd.DataFrame(data)
  df['time'] = pd.to_datetime(df['time'])

  return df

osmtracker_df = parse_gpx('/content/osmtracker.gpx')
osmtracker_df.head()

In [ ]:
import folium

def plot_points(center, osmtracker_points, gnss_logger_points):
  m = folium.Map(
      location=center,
      zoom_start=18,
  )

  folium.PolyLine(osmtracker_points, color="black", weight=2.5, opacity=0.7).add_to(m)
  folium.PolyLine(gnss_logger_points, color="orange", weight=5.5, opacity=0.5).add_to(m)

  return m

osmtracker_points = osmtracker_df[['lat', 'lon']].values.tolist()
gnss_logger_points = gnss_logger_df[gnss_logger_df['Provider'] == 'GPS'][['LatitudeDegrees', 'LongitudeDegrees']].values.tolist()

plot_points(
    (osmtracker_df['lat'].mean(), osmtracker_df['lon'].mean()),
    osmtracker_points,
    gnss_logger_points,
)